In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_impact_of_lmbda_runs(acq):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "xcn0phan",
        "config.amplitude": 1,
        "config.dim": 8,
        "config.kernel": "Matern52",
        "config.lengthscale": 0.1,  
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        summary_metric_keys = ["global optimum value"]
        summary_history = run.scan_history(keys = summary_metric_keys, page_size=1_000_000_000)
        metrics.update({k: [d[k] for d in summary_history] for k in summary_metric_keys})
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [2]:
acquisition_functions = {
    'Gittins_Lambda_1':'Gittins_Lambda1',
    'Gittins_Lambda_01':'Gittins_Lambda01',
    'Gittins_Lambda_001':'Gittins_Lambda001',
    'Gittins_Lambda_0001':'Gittins_Lambda0001',
    'Gittins_Lambda_00001':'Gittins_Lambda00001'
    }

In [3]:
grouped_runs = {(a): load_impact_of_lmbda_runs(a) for a in acquisition_functions.keys()}

0it [00:00, ?it/s]
100%|██████████| 256/256 [04:11<00:00,  1.02it/s]


In [4]:
for a in acquisition_functions.keys():
    config_and_metrics_per_seed = grouped_runs[a]

    cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
    best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
    # Handling potential empty arrays
    if cumulative_cost_per_seed.size == 0 or best_observed_per_seed.size == 0:
        continue  # Skip this iteration if there's no data
    global_optimum_per_seed = np.array([m['global optimum value'][0] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0])

    regret_per_seed = global_optimum_per_seed - best_observed_per_seed
    print(a, regret_per_seed.shape)

    regret_25 = np.quantile(regret_per_seed, 0.25, axis=1)
    regret_50 = np.quantile(regret_per_seed, 0.5, axis=1)
    regret_75 = np.quantile(regret_per_seed, 0.75, axis=1)

    output = np.stack((cumulative_cost_per_seed.mean(axis=1), regret_25, regret_50, regret_75),axis=-1)

    np.savetxt(f"results/impact_of_lmbda_256/Lmbda_Impact_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

Gittins_Lambda_01 (200, 256)
Gittins_Lambda_001 (200, 256)
Gittins_Lambda_0001 (200, 256)
Gittins_Lambda_00001 (200, 256)
